In [41]:
#Import

#Modelling
import mesa

from mesa import Agent, Model

import solara

#Analysis
from mesa.datacollection import DataCollector
from mesa.space import MultiGrid
from mesa.time import RandomActivation

from mesa.experimental import JupyterViz

from matplotlib.figure import Figure

In [42]:
from typing import Callable, Optional, Type

class RandomActivationByTypeFiltered(mesa.time.RandomActivationByType):
    """
    A scheduler that overrides the get_type_count method to allow for filtering
    of agents by a function before counting.

    Example:
    >>> scheduler = RandomActivationByTypeFiltered(model)
    >>> scheduler.get_type_count(AgentA, lambda agent: agent.some_attribute > 10)
    """

    def get_type_count(
        self,
        type_class: Type[mesa.Agent],
        filter_func: Optional[Callable[[mesa.Agent], bool]] = None,
    ) -> int:
        """
        Returns the current number of agents of certain type in the queue
        that satisfy the filter function.
        """
        count = 0
        for agent in self.agents_by_type[type_class].values():
            if filter_func is None or filter_func(agent):
                count += 1
        return count

In [43]:
#Model

class BrainModel(Model):
    def __init__(self, Nn, Na, width, height):
        super().__init__()
        self.num_neurons = Nn
        self.num_antibody = Na
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivationByTypeFiltered(self)
        self.running = True
        self.step_count = 0
        self.datacollector = DataCollector(
            model_reporters={
            "Total surface NMDARs": compute_tot_nmdar,
            "Number of Antibodies": lambda m: m.schedule.get_type_count(AntibAgent)   
        },
        agent_reporters={
            "Surface NMDAR Conc.": "NMDAR",
            "Agent type": "type"
        }, 
    )
        
    # Create Neurons
        for unique_id in range(self.num_neurons):
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            
            neuron = NeuronAgent(unique_id, self)
            self.grid.place_agent(neuron, (x, y))
            self.schedule.add(neuron)

    #Create Antibodies
        for i in range(self.num_antibody):
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            
            antib = AntibAgent((i,x,y), self)
            self.grid.place_agent(antib, (x, y))
            self.schedule.add(antib) 

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()
        self.step_count += 1

#Agents

class NeuronAgent(Agent):
    def __init__(self, unique_id, model):
        # run the __init__ method of the parent class
        super().__init__(unique_id, model)
        # additional initialization for the derived class
        self.NMDAR = 10
        self.type = "Neuron" 

    def step(self):
        if self.model.step_count % 100 == 0 and self.NMDAR < 10:
            self.NMDAR += 1

class AntibAgent(Agent):
    def __init__(self, unique_id, model):
        # run the __init__ method of the parent class
        super().__init__(unique_id, model)
        # additional initialization for the derived class
        self.NMDAR = 0
        self.type = "Antibody"
        self.energy = 100         

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,  # current position
            moore=True,  # including step on the diagonal
            include_center=False,  # to remain at current position is not allowed
        )
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def step(self):
        self.move()
        self.energy -= 1

        #if there is a neuron nearby, internalise NMDAR
        x, y = self.pos
        this_cell = self.model.grid.get_cell_list_contents([self.pos])
        neuron = [obj for obj in this_cell if isinstance(obj, NeuronAgent)]
        if len(neuron) > 0:
             affected_neuron = self.random.choice(neuron)

             # Internalise NMDAR, along with itself
             affected_neuron.NMDAR -= 1

             self.model.grid.remove_agent(self)
             self.model.schedule.remove(self)
        
        #Death from time
        if self.energy < 0:
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)


def compute_tot_nmdar(model):
    
    agent_nmdars =  [agent.NMDAR for agent in model.schedule.agents]

    tot = sum(agent_nmdars)

    return tot    


In [44]:
model = BrainModel(50, 50, 100, 100)

for i in range(90):
    model.step()

In [45]:
agent_df = model.datacollector.get_agent_vars_dataframe()
model_df = model.datacollector.get_model_vars_dataframe()

agent_df

Surface NMDAR Conc. Agent type
Step AgentID                                     
0    0                              10     Neuron
     1                              10     Neuron
     2                              10     Neuron
     3                              10     Neuron
     4                              10     Neuron
...                                ...        ...
89   (43, 51, 80)                    0   Antibody
     (44, 82, 38)                    0   Antibody
     (46, 39, 25)                    0   Antibody
     (48, 63, 3)                     0   Antibody
     (49, 50, 47)                    0   Antibody

[8206 rows x 2 columns]

In [46]:
model_df

,Total surface NMDARs,Number of Antibodies
0,500,50
1,500,50
2,499,49
3,498,48
4,497,47
...,...,...
85,490,40
86,490,40
87,490,40
88,490,40


In [47]:
"""
def agent_portrayal(agent):
    size = 1
    color = "tab:blue"

    if isinstance(agent, AntibAgent):
        size = 0.5
        color = "tab:orange"
    
    return {"size:":size, "color": color}

model_params = {
    "Nn": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of Neurons",
        "min": 50,
        "max": 200,
        "step": 1,
    },
     "Na": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of Antibodies",
        "min": 50,
        "max": 500,
        "step": 1,
    },
    "width":100,
    "height":100,
}


page = JupyterViz(
    BrainModel,
    model_params,
    measures=["Total surface NMDARs", "Number of Antibodies"],
    name = "Brain Model",
    agent_portrayal=agent_portrayal,
)

page
"""

Cannot show ipywidgets in text

In [48]:
"""

def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "blue",
                 "r": 0.5}
    
    if isinstance(agent, AntibAgent):
        portrayal["Color"] = "red"
        portrayal["Layer"] = 1
        portrayal["r"] = 0.2

    return portrayal

grid = mesa.visualization.CanvasGrid(agent_portrayal, 100, 100, 720, 720)
nNMDAR = mesa.visualization.ChartModule(
    [{"Label": "Number of active NMDAR", "Color":"Black"}], data_collector_name="Total surface NMDARs"
)
nAb = mesa.visualization.ChartModule(
    [{"Label": "Number of Antibodies", "Color":"Red"}], data_collector_name="Number of Antibodies"
)

server = mesa.visualization.ModularServer(
    BrainModel, [grid, nNMDAR, nAb], "Brain Model", {"Nn":50, "Na":500, "width":100, "height":100,}
)

server.port = 8521
server.launch()

"""

'\n\ndef agent_portrayal(agent):\n    portrayal = {"Shape": "circle",\n                 "Filled": "true",\n                 "Layer": 0,\n                 "Color": "blue",\n                 "r": 0.5}\n    \n    if isinstance(agent, AntibAgent):\n        portrayal["Color"] = "red"\n        portrayal["Layer"] = 1\n        portrayal["r"] = 0.2\n\n    return portrayal\n\ngrid = mesa.visualization.CanvasGrid(agent_portrayal, 100, 100, 720, 720)\nnNMDAR = mesa.visualization.ChartModule(\n    [{"Label": "Number of active NMDAR", "Color":"Black"}], data_collector_name="Total surface NMDARs"\n)\nnAb = mesa.visualization.ChartModule(\n    [{"Label": "Number of Antibodies", "Color":"Red"}], data_collector_name="Number of Antibodies"\n)\n\nserver = mesa.visualization.ModularServer(\n    BrainModel, [grid, nNMDAR, nAb], "Brain Model", {"Nn":50, "Na":500, "width":100, "height":100,}\n)\n\nserver.port = 8521\nserver.launch()\n\n'